In [0]:
from pyspark.sql.functions import *

# Define expectation rules
expectation_rule = {
    'rule1': 'show_id is NOT Null'
}

expectation_rule_masterdata = {
    'rule1':'show_id is NOT Null',
    'rule2':'type_flag is NOT Null',
}


# Define the gold layer functions
@dlt.table(name="netflix_directors")
@dlt.expect_all_or_drop(expectation_rule)
def gold_function_directors():
    df = spark.readStream \
        .format('delta') \
        .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_directors')
    df.writeStream \
        .option('checkpointLocation', 'abfss://gold@dlnetflix.dfs.core.windows.net/netflix_directors/checkpoint') \
        .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_directors')
    return df

@dlt.table(name="netflix_cast")
@dlt.expect_all_or_drop(expectation_rule)
def gold_function_cast():
    df = spark.readStream \
        .format('delta') \
        .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_cast')
    df.writeStream \
        .option('checkpointLocation', 'abfss://gold@dlnetflix.dfs.core.windows.net/netflix_cast/checkpoint') \
        .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_cast')
    return df

@dlt.table(name="netflix_category")
@dlt.expect_all_or_drop(expectation_rule)
def gold_function_category():
    df = spark.readStream \
        .format('delta') \
        .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_category')
    df.writeStream \
        .option('checkpointLocation', 'abfss://gold@dlnetflix.dfs.core.windows.net/netflix_category/checkpoint') \
        .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_category')
    return df

@dlt.table(name="netflix_countries")
@dlt.expect_all_or_drop(expectation_rule)
def gold_function_countries():
    df = spark.readStream \
        .format('delta') \
        .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_countries')
    df.writeStream \
        .option('checkpointLocation', 'abfss://gold@dlnetflix.dfs.core.windows.net/netflix_countries/checkpoint') \
        .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_countries')
    return df

@dlt.table(name = "netflix_titles")
@dlt.expect_all_or_drop(expectation_rule)
def stg_netflix_titles():
    df = spark.readStream \
        .format('delta') \
        .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_titles')
    return df

@dlt.view(name = "trns_netflix_titles")
def trns_netflix_titles():
    df = spark.readStream.table('Live.netflix_titles')
    df = df.withColumn('type_flag', 
            when(col('type') == 'Movie', lit(1))
            .when(col('type') == 'TV Show', lit(2))
            .otherwise(lit(0)))
    return df

@dlt.table(name = "gold_netflix_titles")
@dlt.expect_all_or_drop(expectation_rule_masterdata)
def netflix_titles():
    df = spark.readStream.table('Live.trns_netflix_titles')
    df.writeStream \
        .option('checkpointLocation', 'abfss://gold@dlnetflix.dfs.core.windows.net/netflix_titles/checkpoint') \
        .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_titles')
    return df


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1747714998621832>, line 17
     14 checkpoint_location = 'abfss://gold@dlnetflix.dfs.core.windows.net/checkpoint'
     16 # Define the gold layer functions
---> 17 @dlt.table(name="netflix_directors")
     18 @dlt.expect_all_or_drop(expectation_rule)
     19 def gold_function_directors():
     20     df = spark.readStream \
     21         .format('delta') \
     22         .load('abfss://silver@dlnetflix.dfs.core.windows.net/netflix_directors')
     23     df.writeStream \
     24         .option('checkpointLocation', checkpoint_location) \
     25         .start('abfss://gold@dlnetflix.dfs.core.windows.net/netflix_directors')

NameError: name 'dlt' is not defined